## Datapane Tutorial - E-Commerce Dashboard

---

This is part of a series of tutorials to help you learn how to use Datapane, and was built by _Kaan Şimşek_.

Before getting started, you will need your API token, which you can find on your [settings page](https://datapane.com/settings). If you are on a private Datapane instance, you will find it in `/settings` on your instance domain (i.e., https://[your-instance].datapane.net/settings).

Once you have you token, add it to the form field in the cell below to login to Datapane.

If you are using Datapane Community, set the `datapane_server_url` as `https://datapane.com`. Alternatively, if you are using a Teams instance, enter the URL of your instance (i.e. https://[your-instance].datapane.net).


In [ ]:
datapane_api_token = ""  # @param {type:"string"}
datapane_server_url = "https://datapane.com"  # @param {type:"string"}

In [ ]:
!pip3 install --quiet datapane

import datapane as dp

dp.login(token=datapane_api_token, server=datapane_server_url)  # Make sure you've set your API token above!

# Import libraries and pull data

In [ ]:
# Import libraries

import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import datetime
import matplotlib.pyplot as plt
import plotly.io as pio
from plotly import tools

In [ ]:
df = pd.read_csv("superstore_orders.csv")
df.head()

In [ ]:
df["Order Date"] = pd.to_datetime(df["Order Date"])
df["Ship Date"] = pd.to_datetime(df["Ship Date"])

daily_quantity = df.groupby("Order Date")["Sales"].sum().reset_index()

trace0 = go.Scatter(x=daily_quantity["Order Date"], y=df["Sales"], name="nflx")
fig0 = go.Figure([trace0])
fig0.update_layout(title={"text": "Total Sales by Day", "x": 0.5, "xanchor": "center"})
fig0.show()

In [ ]:
df["year"] = df["Order Date"].dt.year
df["month"] = df["Order Date"].dt.month
df["dow"] = df["Order Date"].dt.dayofweek
df["day"] = df["Order Date"].dt.day

trace0 = go.Scatter(x=df.groupby("year")["Profit"].sum().index, y=df.groupby("year")["Profit"].sum().values)
trace1 = go.Scatter(x=df.groupby("month")["Profit"].sum().index, y=df.groupby("month")["Profit"].sum().values)
trace2 = go.Scatter(x=df.groupby("dow")["Profit"].sum().index, y=df.groupby("dow")["Profit"].sum().values)
trace3 = go.Scatter(x=df.groupby("day")["Profit"].sum().index, y=df.groupby("day")["Profit"].sum().values)

fig1 = tools.make_subplots(
    rows=4,
    cols=1,
    subplot_titles=(
        "Total Profit by Year",
        "Total Profit by Month",
        "Total Profit by Day of Week",
        "Total Profit by Day of Month",
    ),
)
fig1.append_trace(trace0, 1, 1)
fig1.append_trace(trace1, 2, 1)
fig1.append_trace(trace2, 3, 1)
fig1.append_trace(trace3, 4, 1)
fig1["layout"].update(title="", showlegend=False)

fig1

In [ ]:
trace1 = go.Bar(
    x=df[df.Segment == "Consumer"].groupby("month")["Sales"].sum().index,
    y=df[df.Segment == "Consumer"].groupby("month")["Sales"].sum().values,
    name="Consumer",
)

trace2 = go.Bar(
    x=df[df.Segment == "Corporate"].groupby("month")["Sales"].sum().index,
    y=df[df.Segment == "Corporate"].groupby("month")["Sales"].sum().values,
    name="Corporate",
)

trace3 = go.Bar(
    x=df[df.Segment == "Home Office"].groupby("month")["Sales"].sum().index,
    y=df[df.Segment == "Home Office"].groupby("month")["Sales"].sum().values,
    name="Home Office",
)

layout = go.Layout(title={"text": "Total Sales by Segment", "x": 0.5, "xanchor": "center"}, xaxis=dict(title="Month"))
data = [trace1, trace2, trace3]
fig2 = go.Figure(data=data, layout=layout)
fig2.show()

In [ ]:
trace1 = go.Bar(
    x=df[df.Segment == "Consumer"].groupby("month")["Profit"].sum().index,
    y=df[df.Segment == "Consumer"].groupby("month")["Profit"].sum().values,
    name="Consumer",
)

trace2 = go.Bar(
    x=df[df.Segment == "Corporate"].groupby("month")["Profit"].sum().index,
    y=df[df.Segment == "Corporate"].groupby("month")["Profit"].sum().values,
    name="Corporate",
)

trace3 = go.Bar(
    x=df[df.Segment == "Home Office"].groupby("month")["Profit"].sum().index,
    y=df[df.Segment == "Home Office"].groupby("month")["Profit"].sum().values,
    name="Home-Office",
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    title={"text": "Total Profit by Segment", "x": 0.5, "xanchor": "center"}, xaxis=dict(title="Month"), barmode="stack"
)

fig3 = go.Figure(data=data, layout=layout)
fig3

In [ ]:
def gen_scatter(region, col):
    return go.Scatter(
        x=df[df.Region == region].groupby("month")[col].sum().index,
        y=df[df.Region == region].groupby("month")[col].sum().values,
        name=region,
        mode="markers",
    )


data = [
    gen_scatter("West", "Sales"),
    gen_scatter("East", "Sales"),
    gen_scatter("Central", "Sales"),
    gen_scatter("South", "Sales"),
]

layout = go.Layout(title={"text": "Total Sales by Region", "x": 0.5, "xanchor": "center"}, xaxis=dict(title="Month"))

fig4 = go.Figure(data=data, layout=layout)
fig4

In [ ]:
data = [
    gen_scatter("West", "Profit"),
    gen_scatter("East", "Profit"),
    gen_scatter("Central", "Profit"),
    gen_scatter("South", "Profit"),
]

layout = go.Layout(title={"text": "Total Profit by Region", "x": 0.5, "xanchor": "center"}, xaxis=dict(title="Month"))

fig5 = go.Figure(data=data, layout=layout)
fig5

In [ ]:
data = go.Pie(
    labels=df.groupby("Category")["Sales"].sum().index, values=df.groupby("Category")["Sales"].sum().values, hole=0.2
)
layout = go.Layout(title={"text": "Total Sales by Category", "x": 0.5, "xanchor": "center"})
fig6 = go.Figure(data=data, layout=layout)

fig6

In [ ]:
data = go.Pie(
    labels=df.groupby("Sub-Category")["Profit"].sum().index,
    values=df.groupby("Sub-Category")["Profit"].sum().values,
    hole=0.2,
)
layout = go.Layout(
    title={"text": "Total Profit by Sub-Category", "x": 0.5, "xanchor": "center"}, legend=dict(orientation="h")
)
fig7 = go.Figure(data=data, layout=layout)

fig7

In [ ]:
data = go.Bar(
    y=df.groupby("Category")["Sales"].sum().index, x=df.groupby("Category")["Sales"].sum().values, orientation="h"
)

layout = go.Layout(
    title={"text": "Total Sales by Category", "x": 0.5, "xanchor": "center"}, legend=dict(orientation="h")
)
fig8 = go.Figure(data=data, layout=layout)

fig8

In [ ]:
data = go.Bar(
    y=df.groupby("Sub-Category")["Profit"].sum().index,
    x=df.groupby("Sub-Category")["Profit"].sum().values,
    orientation="h",
)

layout = go.Layout(
    title={"text": "Total Profit by Sub-Category", "x": 0.5, "xanchor": "center"}, legend=dict(orientation="h")
)
fig9 = go.Figure(data=data, layout=layout)
fig9

In [ ]:
top_cities = df.groupby("City")["Quantity"].sum().sort_values(ascending=False)[:10].index

fv = df[df.City.isin(top_cities)].pivot_table(index="City", columns="dow", values="Sales", aggfunc=lambda x: x.mean())

trace = go.Heatmap(
    z=fv.values,
    x=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
    y=fv.index.values,
)

data = [trace]
layout = go.Layout(title={"text": "Total Sales by City", "x": 0.5, "xanchor": "center"})

fig10 = go.Figure(data=data, layout=layout)
fig10

In [ ]:
top_cities = df.groupby("City")["Quantity"].sum().sort_values(ascending=False)[:10].index

fv = df[df.City.isin(top_cities)].pivot_table(index="City", columns="dow", values="Profit", aggfunc=lambda x: x.mean())

trace = go.Heatmap(
    z=fv.values,
    x=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
    y=fv.index.values,
)

data = [trace]
layout = go.Layout(title={"text": "Total Profit by City", "x": 0.5, "xanchor": "center"})

fig11 = go.Figure(data=data, layout=layout)

fig11

# Building a Datapane report

Now that we have a series of plots created using Plotly, we can create a report using Datapane. 

In addition to the visualizations, this report includes Datapane's `HTML` component to display some custom styling.

We are also creating a dropdown using Datpane's `Select` block (see [layout components](https://docs.datapane.com/reports/blocks/layout-pages-and-selects)). This allows the user to choose their desired visualization from a list. 

We have used the `Page` block to add multiple pages to our report, so the user can choose whether to view the forecast or the exploratory data analysis.


In [ ]:
banner_html = """<div style="padding: 10px;display: flex;align-items: center;font-size: 40px;color: #312E81;background: #EEF2FF;">
<img src="https://upload.wikimedia.org/wikipedia/commons/c/c6/Einkaufswagen_%2824140951728%29.jpg" style="margin-right: 1em; max-height: 250px;">
<h1>Superstore Analysis</h1>
</div>
"""
dp.Report(
    dp.Page(
        title="Superstore Dashboard",
        blocks=[
            dp.HTML(banner_html),
            dp.Group(
                dp.Group(
                    "## Sales Analysis",
                    dp.Select(
                        dp.Plot(
                            fig0, label="Sales by Day", caption="This plot shows total sales by day from 2014 to 2017."
                        ),
                        dp.Plot(
                            fig4,
                            label="Sales by Region",
                            caption="This plot shows total sales by west, east, central, south regions.",
                        ),
                        dp.Plot(fig10, label="Sales by City", caption="This plot shows total sales by city."),
                        dp.Plot(
                            fig2,
                            label="Sales by Segment",
                            caption="This plot shows total sales by consumer, corporate, home-office segments.",
                        ),
                        dp.Plot(fig6, label="Sales by Category", caption="This plot shows total sales by category."),
                        dp.Plot(
                            fig8, label="Sales by Subcategory", caption="This plot shows total sales by subcategory."
                        ),
                    ),
                ),
                dp.Group(
                    "## Profit Analysis",
                    dp.Select(
                        dp.Plot(
                            fig1,
                            label="Profit by Day",
                            caption="This plot shows total Profit by day from 2014 to 2017.",
                        ),
                        dp.Plot(
                            fig5,
                            label="Profit by Region",
                            caption="This plot shows total Profit by west, east, central, south regions.",
                        ),
                        dp.Plot(fig11, label="Profit by City", caption="This plot shows total Profit by city."),
                        dp.Plot(
                            fig3,
                            label="Profit by Segment",
                            caption="This plot shows total Profit by consumer, corporate, home-office segments.",
                        ),
                        dp.Plot(fig7, label="Profit by Category", caption="This plot shows total Profit by category."),
                        dp.Plot(
                            fig9, label="Profit by Subcategory", caption="This plot shows total Profit by subcategory."
                        ),
                    ),
                ),
                columns=2,
            ),
        ],
    ),
    type=dp.ReportType.DASHBOARD,
).upload(name="E-Commerce Report", open=True)